# Testing Classes to represent assets - NestSmart -

In [1]:
from icecream import ic

In [2]:
import pytest

In [3]:
import numpy as np
import xarray as xr

class Asset:

    # Will probably need a list of dates for which the levels, payments, etc. correspond (Check pendulum)

    balance_bop = None
    returns = None 
    inflows = None
    outflows = None
    balance_eop = None
    
    def __init__(self,initial_value,growth):
        self.value = xr.DataArray(np.ones(len(growth)+1)) * initial_value


In [ ]:
a = Asset(10,[1,1,1])

In [ ]:
a.value

In [4]:
initial_year = 2000
years_to_simulate = 3
currency = 'GBP'

years = np.arange(initial_year,initial_year+years_to_simulate+1)

In [ ]:
initial_balance =0

return_pct = xr.DataArray([np.nan,0.1,0.05,0.02], 
                           coords={"year": years},
                           attrs = {'long_name':'Investment return','units':'%'}
                          )
inflows = xr.DataArray([np.nan,10.0,20.0,30.0], 
                           coords={"year": years},
                           attrs = {'long_name':'Cash inflows','units':currency}
                          )
outflows = xr.DataArray([np.nan,-00.0,-0.0,-0.0], 
                           coords={"year": years},
                           attrs = {'long_name':'Cash outflows','units':currency}
                          )

In [ ]:
### VERSION WITH INITIAL INVESTMENT ###

### IMPORTANT NOTE: This calculation assumes infows happen at the begining of the year (to make it so that they are spread uniformly across the year divide the returns by 2 - But only the returns on the new investment of that year-)

return_inflows_tmp = xr.DataArray(np.zeros(len(years)), 
                           coords={"year": years},
                           attrs = {'long_name':'Balance at end of period','units':currency}
                          )
#ic(inflows_tmp)

initial_and_inflows = inflows.copy()
initial_and_inflows.loc[years[0]] = initial_balance

for (i,year) in enumerate(years):
    years_left = len(years) - i
    tmp = xr.DataArray(np.ones(years_left) * initial_and_inflows.loc[year].values, 
                       coords={"year": years[i:]},
                       attrs = {'long_name':'Balance at end of period','units':currency}
                      )
    #ic(tmp)
    
    with xr.set_options(arithmetic_join="outer"):
        #ic(((return_pct + 1).cumprod()))
        
        #tmp = tmp * ((return_pct[i:] + 1).cumprod())
        
        cum_ret = xr.concat([return_pct[:i]*0,(return_pct[i:]+1).cumprod()],'year')
        tmp = tmp * cum_ret
        
        #ic(tmp.fillna(0))
        return_inflows_tmp = return_inflows_tmp + tmp.fillna(0)
        
    #ic(return_inflows_tmp)

In [ ]:
return_outflows_tmp = xr.DataArray(np.zeros(len(years)), 
                                  coords={"year": years},
                                  attrs = {'long_name':'Balance at end of period','units':currency}
                                 )
#ic(return_inflows_tmp)

for (i,year) in enumerate(years[1:],start=1):
    years_left = len(years) - i
    tmp = xr.DataArray(np.ones(years_left) * outflows.loc[year].values, 
                       coords={"year": years[i:]},
                       attrs = {'long_name':'Balance at end of period','units':currency}
                      )
    #ic(tmp)
    
    with xr.set_options(arithmetic_join="outer"):
#        tmp = tmp * ((return_pct + 1).cumprod())
        #ic(tmp.fillna(0))
        cum_ret = xr.concat([return_pct[:i]*0,(return_pct[i:]+1).cumprod()],'year')
        tmp = tmp * cum_ret
        return_outflows_tmp = return_outflows_tmp + tmp.fillna(0)
        
    #ic(return_inflows_tmp)

In [ ]:
return_inflows_tmp

In [ ]:
return_inflows_tmp + return_outflows_tmp

## Function and tests

In [5]:
### OLD VERSION - IN CASE I BREAK IT IN THE NEXT CELL -

def calculate_balances (initial_investment, years, inf, outf, returns):
    initial_balance = initial_investment

    return_pct = xr.DataArray(np.concatenate([[np.nan],returns]), 
                           coords={"year": years},
                           attrs = {'long_name':'Investment return','units':'%'}
                          )
    inflows = xr.DataArray(np.concatenate([[np.nan],inf]), 
                           coords={"year": years},
                           attrs = {'long_name':'Cash inflows','units':currency}
                          )
    outflows = xr.DataArray(np.concatenate([[np.nan],outf]), 
                           coords={"year": years},
                           attrs = {'long_name':'Cash outflows','units':currency}
                          )
    
    ### IMPORTANT NOTE: This calculation assumes infows happen at the begining of the year (to make it so that they are spread uniformly across the year divide the returns by 2 - But only the returns on the new investment of that year-)

    return_inflows_tmp = xr.DataArray(np.zeros(len(years)), 
                           coords={"year": years},
                           attrs = {'long_name':'Balance at end of period','units':currency}
                          )

    initial_and_inflows = inflows.copy()
    initial_and_inflows.loc[years[0]] = initial_balance

    for (i,year) in enumerate(years):
        years_left = len(years) - i
        tmp = xr.DataArray(np.ones(years_left) * initial_and_inflows.loc[year].values, 
                           coords={"year": years[i:]},
                           attrs = {'long_name':'Balance at end of period','units':currency}
                          )
    
        with xr.set_options(arithmetic_join="outer"):
            cum_ret = xr.concat([return_pct[:i]*0,(return_pct[i:]+1).cumprod()],'year')
            tmp = tmp * cum_ret
            return_inflows_tmp = return_inflows_tmp + tmp.fillna(0)

    ## OUTFLOWS
    
    return_outflows_tmp = xr.DataArray(np.zeros(len(years)), 
                                  coords={"year": years},
                                  attrs = {'long_name':'Balance at end of period','units':currency}
                                 )

    for (i,year) in enumerate(years[1:],start=1):
        years_left = len(years) - i
        tmp = xr.DataArray(np.ones(years_left) * outflows.loc[year].values, 
                           coords={"year": years[i:]},
                           attrs = {'long_name':'Balance at end of period','units':currency}
                          )
        with xr.set_options(arithmetic_join="outer"):
            cum_ret = xr.concat([return_pct[:i]*0,(return_pct[i:]+1).cumprod()],'year')
            tmp = tmp * cum_ret
            return_outflows_tmp = return_outflows_tmp + tmp.fillna(0)

    return return_inflows_tmp + return_outflows_tmp

In [14]:
def calculate_balances (initial_investment, cash_in, cash_out, returns, years):

    ### IMPORTANT NOTE: This calculation assumes infows happen at the begining of the year (to make it so that they are spread uniformly across the year divide the returns by 2 - But only the returns on the new investment of that year-)

    initial_balance = initial_investment

    return_pct = xr.DataArray(np.concatenate([[np.nan],returns]), 
                           coords={"year": years},
                           attrs = {'long_name':'Investment return','units':'%'}
                          )
    inflows = xr.DataArray(np.concatenate([[np.nan],inf]), 
                           coords={"year": years},
                           attrs = {'long_name':'Cash inflows','units':currency}
                          )
    outflows = xr.DataArray(np.concatenate([[np.nan],outf]), 
                           coords={"year": years},
                           attrs = {'long_name':'Cash outflows','units':currency}
                          )
    
    balance_eop_tmp = xr.DataArray(np.zeros(len(years)), 
                                   coords={"year": years},
                                   attrs = {'long_name':'Balance at end of period','units':currency}
                                  )

    all_cashflows = inflows.copy()
    all_cashflows.loc[years[0]] = initial_balance
    with xr.set_options(arithmetic_join="outer"):
        all_cashflows += outflows.fillna(0)


    for (i,year) in enumerate(years):
        years_left = len(years) - i
        tmp = xr.DataArray(np.ones(years_left) * all_cashflows.loc[year].values, 
                           coords={"year": years[i:]},
                           attrs = {'long_name':'Balance at end of period','units':currency}
                          )
    
        with xr.set_options(arithmetic_join="outer"):
            cum_ret = xr.concat([return_pct[:i]*0,(return_pct[i:]+1).cumprod()],'year')
            #ic(cum_ret)
            tmp = tmp * cum_ret
            #ic(tmp)
            balance_eop_tmp = balance_eop_tmp + tmp.fillna(0)

    return balance_eop_tmp

In [8]:
        initial_investment = 0
        years = [2023,2024,2025,2026,2027]
        inf = [10,5,2,1]
        outf = [-2,-2,-1,-1]
        returns = [0.1,0.05,0.02,0.01]

calculate_balances(initial_investment=initial_investment,
                   years=years,
                   inf=inf,
                   outf=outf,
                   returns=returns)

<xarray.DataArray (year: 5)>
array([ 0.      ,  8.8     , 12.39    , 13.6578  , 13.794378])
Coordinates:
  * year     (year) int64 2023 2024 2025 2026 2027

In [9]:
import ipytest
ipytest.autoconfig()

In [16]:
%%ipytest -qq

class TestCalculateEndingBalances():

    def test_returns_from_initial_investment_calculated_correctly(self):
        initial_investment = 100
        years = [2023,2024,2025,2026,2027]
        inf = [0,0,0,0]
        outf = [0,0,0,0]
        returns = [0.1,0.05,0.02,0.01]

        expected_result = xr.DataArray([100,110,115.5,117.81,118.988], coords={'year':years})

        result = calculate_balances(initial_investment=initial_investment,
                                    years=years,
                                    cash_in=inf,
                                    cash_out=outf,
                                    returns=returns)
    
        xr.testing.assert_allclose(result,expected_result)

    def test_returns_from_inflows_calculated_correctly(self):
        initial_investment = 0
        years = [2023,2024,2025,2026,2027]
        inf = [10,5,2,1]
        outf = [0,0,0,0]
        returns = [0.1,0.05,0.02,0.01]

        expected_result = xr.DataArray([0,11,16.8,19.176,20.3777], coords={'year':years})

        result = calculate_balances(initial_investment=initial_investment,
                                    years=years,
                                    cash_in=inf,
                                    cash_out=outf,
                                    returns=returns)
    
        xr.testing.assert_allclose(result,expected_result)

    def test_returns_from_outflows_calculated_correctly(self):
        initial_investment = 0
        years = [2023,2024,2025,2026,2027]
        inf = [0,0,0,0]
        outf = [-10,-5,-2,-1]
        returns = [0.1,0.05,0.02,0.01]

        expected_result = xr.DataArray([0,-11,-16.8,-19.176,-20.3777], coords={'year':years})

        result = calculate_balances(initial_investment=initial_investment,
                                    years=years,
                                    cash_in=inf,
                                    cash_out=outf,
                                    returns=returns)
    
        xr.testing.assert_allclose(result,expected_result)    

    def test_positive_returns_calculated_correctly(self):
        initial_investment = 100
        years = [2023,2024,2025,2026,2027]
        inf = [10,5,2,1]
        outf = [-2,-2,-1,-1]
        returns = [0.1,0.05,0.02,0.01]

        expected_result = xr.DataArray([100,118.8,127.89,131.467,132.782], coords={'year':years})

        result = calculate_balances(initial_investment=initial_investment,
                                    years=years,
                                    cash_in=inf,
                                    cash_out=outf,
                                    returns=returns)
    
        xr.testing.assert_allclose(result,expected_result)

    def test_negative_returns_calculated_correctly(self):
        initial_investment = 100
        years = [2023,2024,2025,2026,2027]
        inf = [10,5,2,1]
        outf = [-2,-2,-1,-1]
        returns = [-0.1,-0.05,-0.02,-0.01]

        expected_result = xr.DataArray([100,97.2,95.19,94.266,93.323], coords={'year':years})

        result = calculate_balances(initial_investment=initial_investment,
                                    years=years,
                                    cash_in=inf,
                                    cash_out=outf,
                                    returns=returns)
    
        xr.testing.assert_allclose(result,expected_result)

    def test_zero_returns_calculated_correctly(self):
        initial_investment = 100
        years = [2023,2024,2025,2026,2027]
        inf = [10,5,2,1]
        outf = [-2,-2,-1,-1]
        returns = [0,0,0,0]

        expected_result = xr.DataArray([100,108,111,112,112], coords={'year':years})

        result = calculate_balances(initial_investment=initial_investment,
                                    years=years,
                                    cash_in=inf,
                                    cash_out=outf,
                                    returns=returns)
    
        xr.testing.assert_allclose(result,expected_result)



FFF...                                                                                       [100%]
============================================= FAILURES =============================================
______ TestCalculateEndingBalances.test_returns_from_initial_investment_calculated_correctly _______

self = <__main__.TestCalculateEndingBalances object at 0x7ff3506f71a0>

    def test_returns_from_initial_investment_calculated_correctly(self):
        initial_investment = 100
        years = [2023,2024,2025,2026,2027]
        inf = [0,0,0,0]
        outf = [0,0,0,0]
        returns = [0.1,0.05,0.02,0.01]
    
        expected_result = xr.DataArray([100,110,115.5,117.81,118.988], coords={'year':years})
    
        result = calculate_balances(initial_investment=initial_investment,
                                    years=years,
                                    cash_in=inf,
                                    cash_out=outf,
                                    returns=returns)
    
> 

In [ ]:
# TO DELETE
balance_eop_tmp = xr.DataArray(np.ones(len(years)) * initial_balance, 
                           coords={"year": years[0:]},
                           attrs = {'long_name':'Balance at end of period','units':currency}
                          )
balance_eop_tmp = balance_eop_tmp * ((return_pct + 1).cumprod())
investment_returns = balance_eop_tmp.diff('year')

In [ ]:
## TO DELETE ##

return_inflows_tmp = xr.DataArray(np.zeros(len(years)), 
                                  coords={"year": years},
                                  attrs = {'long_name':'Balance at end of period','units':currency}
                                 )
#ic(return_inflows_tmp)

for (i,year) in enumerate(years[1:],start=1):
    years_left = len(years) - i
    tmp = xr.DataArray(np.ones(years_left) * inflows.loc[year].values, 
                       coords={"year": years[i:]},
                       attrs = {'long_name':'Balance at end of period','units':currency}
                      )
    #ic(tmp)
    
    with xr.set_options(arithmetic_join="outer"):
        tmp = tmp * ((return_pct + 1).cumprod())
        #ic(tmp.fillna(0))
        return_inflows_tmp = return_inflows_tmp + tmp.fillna(0)
        
    #ic(return_inflows_tmp)

In [ ]:
balance_eop_tmp + return_inflows_tmp + return_outflows_tmp

In [ ]:
total_change_pct = return_pct + inflows_pct + outflows_pct
total_change_pct.attrs['long_name'] = 'Total change in period'
total_change_pct.attrs['units'] = '%'

In [ ]:
total_change_pct

In [ ]:
# Next steps
#     - Write automated tests for balance calculation
#     - Once we have the evolution of balances, calculate taxable, non-taxable income, capital gains income, etc.